In [1]:
import designMethods.en_13001_3_3 as en
from output import create_output_file
import ipywidgets as widgets
from ipyfilechooser import FileChooser
import pathlib

In [2]:
# !pip install voila
# !jupyter serverextension enable voila --sys-prefix

In [3]:
en_13001_input = en.EN_input()
en_13001_computed = en.Computed_data()
gen_vars = {"num_run": 1}
direction = 1

In [4]:
def read_file():
    en_13001_input.load_gp_input(fc.selected, "configuration")

    # load material and en 13001 parameters
    en_13001_input.load_material_input_check(fc.selected, "rail_materials", "wheel_materials")
    en_13001_input.load_parameter_input(fc.selected, "Input_variables")

    # check for input errors and drop faulty configurations
    en_13001_input.perform_error_checks()
    en_13001_input.drop_error_configs()

    # precompute factors and material parameters
    en_13001_input.parameters.compute_f_f3()
    en_13001_input.parameters.compute_contact_and_f_1()
    en_13001_input.set_materials_and_geometry()

In [5]:
def init_gps(change):
    en_13001_input.config = config_dropdown.value
    gp_configs = {"m1": {1: "m1l", -1: "m1r"}, "m2": {1: "m2"}}
    gp_config = gp_configs[en_13001_input.config][direction]
    parts = ["wf", "wr", "r"]
    en_13001_computed.get_gps_kc(gp_config, parts)
    en_13001_input.config_loaded = True
    msg_box.value = "Initialized GPs"

In [6]:
def start_computation(change):
    if en_13001_input.parameters.loaded is None:
        msg_box.value = "Please upload an excel file"
        return

    if en_13001_input.config_loaded is None:
        init_gps(None)
    msg_box.value = "Start Computation"
    en_13001_input.recompute_gp_data(en_13001_input.config)
    en_13001_computed.predict_kc(en_13001_input.gp_input.norm)
    en_13001_computed.compute_F_sd_f_all(en_13001_input.gp_input.raw, en_13001_input.config, direction)
    en_13001_computed.predict_travelled_dist(en_13001_input.parameters.data)

    # create computation instance and compute configs
    en_computation = en.Computation(en_13001_input, en_13001_computed)
    en_computation.compute_pre_F_rd_all()
    en_computation.compute_F_rd_all()
    en_computation.compute_proofs_all()

    # reults output
    en_13001_input.prepare_for_output()
    en_computation.load_results_all()
    msg_box.value = "Done"
    gen_vars["outname"] = pathlib.Path(fc.selected_path) / f"output_no{gen_vars['num_run']}.xlsx"
    create_output_file(en_computation, en_13001_input, gen_vars["outname"])
    msg_box.value = f"Generated output file: output_no{gen_vars['num_run']}.xlsx"


In [7]:
fc = FileChooser()
fc.register_callback(read_file)
mode_dropdown = widgets.Dropdown(description="Computation Mode", options=["proof", "min_diameter"], value="proof")
config_dropdown = widgets.Dropdown(description="SC Configuration", options=["m1", "m2"], value="m1")
config_button = widgets.Button(description="Update Configuration")
config_button.on_click(init_gps)
start_button = widgets.Button(description="Start computation")
start_button.on_click(start_computation)

msg_box = widgets.Text(value="", description="Message Box")

In [8]:
# VBox([widgets.Label("MARS Demonstrator"),
#         mode_dropdown, config_dropdown, config_button, fc, msg_box, start_button])

display(mode_dropdown)
display(config_dropdown)
display(config_button)
display(fc)
display(msg_box)
display(start_button)

Dropdown(description='Computation Mode', options=('proof', 'min_diameter'), value='proof')

Dropdown(description='SC Configuration', options=('m1', 'm2'), value='m1')

Button(description='Update Configuration', style=ButtonStyle())

FileChooser(path='C:\Dokumente\MARS\mars-softwaredemonstrator\marsDemonstrator', filename='', title='HTML(valu…

Text(value='', description='Message Box')

Button(description='Start computation', style=ButtonStyle())